In [20]:
import matplotlib.pyplot as plt
import numpy as np

from neuropacks import NHP
from pyuoi.decomposition import CUR
from scipy.io import loadmat
from sklearn.linear_model import LinearRegression

%matplotlib inline

In [2]:
nhp = NHP(data_path='/Users/psachdeva/data/NHP/indy_20160411_01.mat')

In [3]:
Y = nhp.get_response_matrix(
    bin_width=0.25, region='M1'
)
cursor_pos = nhp.get_binned_positions(bin_width=0.25)

In [4]:
max_k = 40
cur = CUR(
    n_boots=20,
    max_k=max_k,
    boots_frac=0.8
)
cur.fit(Y)

CUR(algorithm='randomized', boots_frac=0.8, max_k=40, n_boots=20, n_iter=5,
  random_state=None, tol=0.0)

In [16]:
cur.columns_.shape


(16,)

In [35]:
_, _, V = np.linalg.svd(Y)

cur_flags = CUR.column_select(V[:max_k].T, 13)
n_cur = np.count_nonzero(cur_flags)

In [37]:
np.argwhere(cur_flags).ravel().shape

(15,)

In [38]:
normal_indices = np.argwhere(cur_flags).ravel()[:16]

In [47]:
ols = LinearRegression()
ols.fit(Y[:1500, cur.columns_], cursor_pos[:1500, 1])
print(ols.score(Y[1500:, cur.columns_], cursor_pos[1500:, 1]))
ols.fit(Y[:2500, normal_indices], cursor_pos[:2500, 1])
print(ols.score(Y[2500:, normal_indices], cursor_pos[2500:, 1]))

0.2538812206178086
0.11303730114454758


In [41]:
normal_indices

array([ 14,  29,  42,  45,  51,  72,  74,  86,  89,  99, 111, 139, 141,
       147, 171])

In [44]:
cur.columns_

array([  0,   3,  38,  41,  42,  45,  54,  68,  74,  86, 104, 111, 132,
       171, 174, 176])

In [45]:
Y.shape

(3813, 192)